In [3]:
import pschitt.emission as em
import pschitt.vizualisation as viz
import pschitt.sky_objects as sky
import pschitt.geometry as geo
import pschitt.camera_image as ci
import numpy as np
import matplotlib.pyplot as plt
import math
import pschitt.dataset as ds
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [5]:
import matplotlib
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

font = {'size'   : 20}
matplotlib.rc('font', **font)

In [ ]:
shower = sky.shower()

shower.impact_point = np.array([100,100,0])

#shower direction
shower.alt = math.radians(70)
shower.az = math.radians(0)

#pointing direction off-axis
talt = math.radians(70)
taz = math.radians(0)

#shower parameters
shower_top = 12000
shower_length = 8000
shower_width = 200

shower.number_of_particles = int(1e5)

shower.gaussian_ellipsoide(shower_top, shower_length, shower_width)

tel_normal = geo.altaz_to_normal(talt,taz)
# tel1 = geo.Telescope([0,0,0],tel_normal)
# tel2 = geo.Telescope([200,0,0],tel_normal)
# tel3 = geo.Telescope([400,0,0],tel_normal)
# tel4 = geo.Telescope([800,0,0],tel_normal)
# tel5 = geo.Telescope([0,400,0],tel_normal)

# alltel = [tel1, tel2, tel3, tel4, tel5]

alltel = geo.load_telescopes(ds.get('CTA_paranal.txt'), normal=tel_normal)

trigger_intensity = 20.
noise = 0

shower.particles_angular_emission_profile = em.angular_profile.lgdt06
shower.particles_angular_emission_profile_kwargs = {'eta':1}

ci.array_shower_imaging(shower, alltel, noise)

100 telescopes loaded


In [ ]:
def plot_array(telescope_array, ax=None, **kwargs):
    """
    Plot a map of the telescopes array

    Parameters
    ----------
    telescope_array: list of telescopes classes
    """

    ax = plt.gca() if ax is None else ax

    for tel in telescope_array:
        ax.scatter(tel.mirror_center[0], tel.mirror_center[1], **kwargs)
        ax.annotate(str(tel.id), (tel.mirror_center[0] + 20, tel.mirror_center[1] + 20))
    ax.axis('equal')
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")

    return ax

In [ ]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)
# widgets.IntSlider(min=0,max=len(alltel)-1,step=1,value=0)
@interact(i=widgets.Dropdown(options=np.arange(1,len(alltel)), description="Telescope"))
def f(i): 
    fig, axes = plt.subplots(1,2,figsize=(42,20), gridspec_kw = {'width_ratios':[5, 3]});
    
    tel_id = i-1
    
    axes[0] = plot_array(alltel, ax=axes[0], color='grey')
    ax = viz.display_camera_image(alltel[i], s=140, ax=axes[1]);
    axes[0] = plot_array([alltel[tel_id]], ax=axes[0], color='red')
    # viz.plot_array_reconstructed
    axes[1].set_title("Telescope {}".format(alltel[tel_id].id), fontsize=20);
    axes[0].scatter(shower.impact_point[0], shower.impact_point[1], color='blue', label='Impact point', marker="x", s=300)
    axes[0].legend()
    
